# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 05**: Data pipeline with Neo4j

**Date**: October 2nd 2025

**Student Name**:

**Professor**: Pablo Camarillo Ramirez

# Dataset description

# Data ingestion

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col


spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://87a7c16492c8:7077") \
    .config("spark.jars.packages", "org.neo4j:neo4j-connector-apache-spark_2.13:5.3.10_for_spark_3") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.neo4j#neo4j-connector-apache-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-de3eb2b0-37f4-4a79-826a-d3ffc8ab4fe4;1.0
	confs: [default]
	found org.neo4j#neo4j-connector-apache-spark_2.13;5.3.10_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.13_common;5.3.10_for_spark_3 in central
	found org.neo4j#caniuse-core;1.3.0 in central
	found org.neo4j#caniuse-api;1.3.0 in central
	found org.jetbrains.kotlin#kotlin-stdlib;2.1.20 in central
	found org.jetbrains#annotations;13.0 in central
	found org.neo4j#caniuse-neo4j-detection;1.3.0 in central
	found org.neo4j.driver#neo4j-java-driver-slim;4.4.21 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found io.netty#netty-handler;4.1.

In [4]:
# Build schema
# Import your module
from eduardo_navarro.spark_utils import SparkUtils
patients_schema = SparkUtils.generate_schema([("patient_id", "string"), ("name", "string"), ("date_of_birth","date"),("gender", "string"), ("blood_type","string")])
doctors_schema = SparkUtils.generate_schema([("doctor_id", "string"), ("name", "string"), ("license_number", "string"), ("years_experience","int")])
attends_schema = SparkUtils.generate_schema([("uid", "string"), ("uid2", "string")])

base_path = "/opt/spark/work-dir/data/medical/"
df_patients = spark.read \
           .option("header", "true") \
           .schema(patients_schema) \
           .csv(base_path + "patients/patients_Dgraph.csv")
df_doctors = spark.read \
           .option("header", "true") \
           .schema(doctors_schema) \
           .csv(base_path + "doctors/doctors_Dgraph.csv")
df_attends = spark.read \
    .option("header", "true") \
    .schema(attends_schema) \
    .csv(base_path + "attends/attends.csv")



df_patients.show()
df_doctors.show()
df_attends.show()

+----------+----------------+-------------+------+----------+
|patient_id|            name|date_of_birth|gender|blood_type|
+----------+----------------+-------------+------+----------+
|     P0006|    Sophia Brown|   1966-02-20|     M|        B-|
|     P0004|      Emma White|   1976-03-01|     M|        A-|
|     P0005|  David Martinez|   1989-11-20|     M|        B+|
|     P0001|        John Doe|   1991-09-19|     M|        B-|
|     P0007|Michael Anderson|   1972-04-09|     M|        B+|
|     P0003|     Bob Johnson|   1960-09-25|     F|        B+|
|     P0002|     Alice Smith|   1984-04-30|     M|        A+|
|     P0008|   Olivia Thomas|   1981-09-21|     F|        A+|
|     P0009|    James Harris|   1956-01-16|     F|        B-|
|     P0010|Charlotte Wilson|   1994-09-24|     F|        O-|
+----------+----------------+-------------+------+----------+

+---------+-----------------+--------------+----------------+
|doctor_id|             name|license_number|years_experience|
+------

# Transformations

In [6]:
# Add the code for your transformations to create nodes and edges DataFrames HERE
patients_nodes = df_patients.select(
    col("patient_id").alias("id"),
    col("name"),
    col("gender")
).dropDuplicates(["id"])
patients_nodes.show()

doctors_nodes = df_doctors.select(
    col("doctor_id").alias("id"),
    col("name"),
    col("license_number")
).dropDuplicates(["id"])
doctors_nodes.show()

attends_edges = df_attends.select(
    col("uid").alias("src"),   # source node (User)
    col("uid2").alias("dst"),   # destination node (Product)
)
attends_edges.show()

+-----+----------------+------+
|   id|            name|gender|
+-----+----------------+------+
|P0001|        John Doe|     M|
|P0002|     Alice Smith|     M|
|P0003|     Bob Johnson|     F|
|P0004|      Emma White|     M|
|P0005|  David Martinez|     M|
|P0006|    Sophia Brown|     M|
|P0007|Michael Anderson|     M|
|P0008|   Olivia Thomas|     F|
|P0009|    James Harris|     F|
|P0010|Charlotte Wilson|     F|
+-----+----------------+------+

+-----+-----------------+--------------+
|   id|             name|license_number|
+-----+-----------------+--------------+
|D0001|       Liam Davis|     CAR837492|
|D0002|    Sophia Miller|     DER374862|
|D0003|      Noah Garcia|     NEU826374|
|D0004| Amelia Rodriguez|     ORT982374|
|D0005|      Ethan Lopez|     PED198374|
|D0006|Isabella Gonzalez|     ONC264738|
|D0007|      Mason Perez|     OPH736481|
|D0008|       Mia Taylor|     PSY837465|
|D0009|      Lucas Clark|     END374920|
|D0010|    Harper Walker|     GAS746291|
+-----+-----------

# Writing Data in Neo4j

In [7]:
# Add the code to write a graph from PySpark's DataFrames to Neo4j
neo4j_url = "bolt://neo4j-iteso:7687"
neo4j_user = "neo4j"
neo4j_passwd = "neo4j@1234"

patients_nodes.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("labels", ":Patient") \
  .option("node.keys", "id") \
  .save()

print(f"{patients_nodes.count()} Patient nodes written to Neo4j")

doctors_nodes.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("labels", ":Doctor") \
  .option("node.keys", "id") \
  .save()

print(f"{doctors_nodes.count()} Doctor nodes written to Neo4j")


attends_edges.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("relationship", "ATTENDS") \
  .option("relationship.save.strategy", "keys") \
  .option("relationship.source.labels", ":Doctor") \
  .option("relationship.source.save.mode", "match") \
  .option("relationship.source.node.keys", "src:id") \
  .option("relationship.target.labels", ":Patient") \
  .option("relationship.target.save.mode", "match") \
  .option("relationship.target.node.keys", "dst:id") \
  .save()

print(f"{attends_edges.count()} ATTENDS relationships written to Neo4j")


10 Patient nodes written to Neo4j
10 Doctor nodes written to Neo4j
14 ATTENDS relationships written to Neo4j


# Read and Query Graphs with PySpark

In [11]:
# Add the code to read a data frame from Neo4J and run a simple query to verify 
cypher_df = spark.read \
    .format("org.neo4j.spark.DataSource") \
    .option("url", neo4j_url) \
    .option("authentication.basic.username", neo4j_user) \
    .option("authentication.basic.password", neo4j_passwd) \
    .option("query",
            """
            MATCH (d:Doctor)-[:ATTENDS]->(p:Patient)
            WHERE p.name = 'John Doe'
            RETURN d.id AS doctor_id, d.name AS doctor_name, p.id AS patient_id, p.name AS patient_name 
            """) \
    .load()

cypher_df.show()


+---------+-----------+----------+------------+
|doctor_id|doctor_name|patient_id|patient_name|
+---------+-----------+----------+------------+
|    D0009|Lucas Clark|     P0001|    John Doe|
|    D0001| Liam Davis|     P0001|    John Doe|
+---------+-----------+----------+------------+



In [12]:
sc.stop()